<a href="https://colab.research.google.com/github/zakariapast/tensorflow-lab/blob/main/tests/dp_training_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ✅ Install dependencies
!pip install -U tensorflow==2.15 tensorflow-privacy==0.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/

In [1]:
# ✅ Imports
import tensorflow as tf
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# ✅ Load and preprocess MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0
x_train = x_train[..., tf.newaxis]
x_test  = x_test[..., tf.newaxis]

print("Training shape:", x_train.shape)
print("Test shape:", x_test.shape)


11490434/11490434 [==============================] - 0s 0us/step
Training shape: (60000, 28, 28, 1)
Test shape: (10000, 28, 28, 1)


In [3]:
# ✅ Build model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])


In [4]:
# ✅ Compile with Differential Privacy
dp_optimizer = DPKerasAdamOptimizer(
    l2_norm_clip=1.0,
    noise_multiplier=1.1,
    num_microbatches=250,
    learning_rate=0.001
)

model.compile(
    optimizer=dp_optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction=tf.keras.losses.Reduction.NONE
),
    metrics=['accuracy']
)


In [5]:
# ✅ Train the model
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=250,
    validation_split=0.1
)


Epoch 1/5
216/216 [==============================] - 104s 479ms/step - loss: 1.0931 - accuracy: 0.6910 - val_loss: 0.4963 - val_accuracy: 0.8523
Epoch 2/5
216/216 [==============================] - 104s 480ms/step - loss: 0.5109 - accuracy: 0.8408 - val_loss: 0.3572 - val_accuracy: 0.8920
Epoch 3/5
216/216 [==============================] - 103s 478ms/step - loss: 0.4432 - accuracy: 0.8679 - val_loss: 0.3316 - val_accuracy: 0.9030
Epoch 4/5
216/216 [==============================] - 109s 505ms/step - loss: 0.4253 - accuracy: 0.8788 - val_loss: 0.3171 - val_accuracy: 0.9088
Epoch 5/5
216/216 [==============================] - 106s 493ms/step - loss: 0.4164 - accuracy: 0.8863 - val_loss: 0.3128 - val_accuracy: 0.9133


In [6]:
dp_history = history.history